In [2]:
import os
import pandas as pd

root_directory = '/Users/divya/Documents/Data_collection/final_correct_dataset_with_default//'

dataframes = []

# Walk through the root directory and its subdirectories
for dirpath, dirnames, filenames in os.walk(root_directory):
    
    for filename in filenames:
        if filename.endswith('.csv'):
            file_path = os.path.join(dirpath, filename)
            df = pd.read_csv(file_path)
            dataframes.append(df)

# Concatenate all DataFrames into one big DataFrame
big_dataframe = pd.concat(dataframes, ignore_index=True)
big_dataframe

,class,time,fsr1,fsr2,fsr3,fsr4,angle,accx,accy,accz,...,gyro2,gyro3,orient1,orient2,orient3,acc2x,acc2y,acc2z,participant,serialnumber
0,2,194625.0,606.0,9.0,7.0,2.0,1499.0,-0.64,0.27,-0.04,...,0.06,-0.87,333.62,63.06,60.69,8.10,-3.59,2.12,9,3022
1,2,194852.0,1585.0,17.0,10.0,1.0,1331.0,-0.44,0.45,0.00,...,0.04,-0.83,341.62,69.50,49.25,8.75,-2.06,2.24,9,3022
2,2,195068.0,41.0,17.0,0.0,1.0,1263.0,1.69,0.45,-0.09,...,0.16,-0.32,350.44,73.69,24.31,11.11,-0.67,2.40,9,3022
3,2,195287.0,121.0,7.0,2.0,2.0,1688.0,0.41,0.32,-0.14,...,-0.10,0.33,349.81,73.06,25.12,10.10,-0.82,2.50,9,3022
4,2,195509.0,2583.0,13.0,19.0,1.0,1587.0,1.32,0.38,-0.13,...,-0.16,3.04,329.87,59.13,64.25,9.74,-4.15,2.04,9,3022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53221,6,750297.0,30.0,2217.0,1.0,5.0,2490.0,-0.71,-0.46,-0.12,...,0.11,-0.33,56.88,10.81,82.12,2.00,-9.88,1.33,14,5144
53222,6,750517.0,43.0,1832.0,2.0,1.0,2539.0,0.05,0.09,0.03,...,-0.01,-0.03,59.13,12.88,82.37,2.25,-9.38,1.30,14,5144
53223,6,750735.0,22.0,1484.0,1.0,2.0,2501.0,-0.03,0.04,-0.02,...,-0.00,-0.05,59.94,13.63,82.12,2.28,-9.39,1.27,14,5144
53224,6,750956.0,43.0,988.0,0.0,3.0,2464.0,-0.26,-0.13,-0.09,...,-0.02,-0.14,61.63,15.31,82.19,2.74,-9.51,1.21,14,5144


In [ ]:
# Normalisation - It reduced accuracy, so need not to run this
column_min_max_values = {
    'angle': (big_dataframe['angle'].min(), big_dataframe['angle'].max()),
    'fsr1': (big_dataframe['fsr1'].min(), big_dataframe['fsr1'].max()),
    'fsr2': (big_dataframe['fsr2'].min(), big_dataframe['fsr2'].max()),
    'fsr3': (big_dataframe['fsr3'].min(), big_dataframe['fsr3'].max()),
    'fsr4': (big_dataframe['fsr4'].min(), big_dataframe['fsr4'].max()),
    'acc2x':(-15.0,15.0),
    'acc2y':(-15.0,15.0),
    'acc2z':(-15.0,15.0),
    'gyro1':(-6.0,6.0),
    'gyro2':(-6.0,6.0),
    'gyro3':(-6.0,6.0),
     #'orient1': (0, 350),
     #'orient2': (0, 100),
     #'orient3': (0, 100),
}

root_directory = '/Users/divya/Documents/Data_collection/final_correct_dataset_with_default//'

dataframes = []
for dirpath, dirnames, filenames in os.walk(root_directory):
    for filename in filenames:
        if filename.endswith('.csv'):
            file_path = os.path.join(dirpath, filename)
            df = pd.read_csv(file_path)
            normalized_data = df.copy()
            for column, (min_val, max_val) in column_min_max_values.items():
                normalized_data[column] = (df[column] - min_val) / (max_val - min_val)
            dataframes.append(normalized_data)

big_dataframe = pd.concat(dataframes, ignore_index=True)
big_dataframe

In [3]:
# This block is to select the 2 out of 4 FSRs for all the participants
grouped_df = big_dataframe.groupby('participant')

# Mapping of participant to the columns to copy
participant_column_mapping = {
    1: ['fsr2', 'fsr4'],
    2: ['fsr2', 'fsr4'],
    3: ['fsr2', 'fsr4'],
    4: ['fsr2', 'fsr4'],
    5: ['fsr2', 'fsr4'],
    6: ['fsr2', 'fsr4'],
    9: ['fsr2', 'fsr4'],
    10: ['fsr2', 'fsr3'],
    11: ['fsr2', 'fsr4'],
    12: ['fsr1', 'fsr3'],
    13: ['fsr2', 'fsr4'],
    14: ['fsr2', 'fsr3'],

}
big_dataframe['FSR1'] = ''
big_dataframe['FSR2'] = ''

for participant_id, cols_to_copy in participant_column_mapping.items():
    mask = big_dataframe['participant'] == participant_id
    big_dataframe.loc[mask, 'FSR1'] += big_dataframe.loc[mask, cols_to_copy[0]].astype(str) + ','
    big_dataframe.loc[mask, 'FSR2'] += big_dataframe.loc[mask, cols_to_copy[1]].astype(str) + ','

big_dataframe['FSR1'] = big_dataframe['FSR1'].str.rstrip(',')
big_dataframe['FSR2'] = big_dataframe['FSR2'].str.rstrip(',')

big_dataframe=big_dataframe.drop(columns=['fsr1','fsr2','fsr3','fsr4'])
big_dataframe


,class,time,angle,accx,accy,accz,gyro1,gyro2,gyro3,orient1,orient2,orient3,acc2x,acc2y,acc2z,participant,serialnumber,FSR1,FSR2
0,2,194625.0,1499.0,-0.64,0.27,-0.04,0.06,0.06,-0.87,333.62,63.06,60.69,8.10,-3.59,2.12,9,3022,9.0,2.0
1,2,194852.0,1331.0,-0.44,0.45,0.00,0.00,0.04,-0.83,341.62,69.50,49.25,8.75,-2.06,2.24,9,3022,17.0,1.0
2,2,195068.0,1263.0,1.69,0.45,-0.09,0.03,0.16,-0.32,350.44,73.69,24.31,11.11,-0.67,2.40,9,3022,17.0,1.0
3,2,195287.0,1688.0,0.41,0.32,-0.14,-0.02,-0.10,0.33,349.81,73.06,25.12,10.10,-0.82,2.50,9,3022,7.0,2.0
4,2,195509.0,1587.0,1.32,0.38,-0.13,-0.13,-0.16,3.04,329.87,59.13,64.25,9.74,-4.15,2.04,9,3022,13.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53221,6,750297.0,2490.0,-0.71,-0.46,-0.12,-0.20,0.11,-0.33,56.88,10.81,82.12,2.00,-9.88,1.33,14,5144,2217.0,1.0
53222,6,750517.0,2539.0,0.05,0.09,0.03,0.06,-0.01,-0.03,59.13,12.88,82.37,2.25,-9.38,1.30,14,5144,1832.0,2.0
53223,6,750735.0,2501.0,-0.03,0.04,-0.02,0.00,-0.00,-0.05,59.94,13.63,82.12,2.28,-9.39,1.27,14,5144,1484.0,1.0
53224,6,750956.0,2464.0,-0.26,-0.13,-0.09,-0.01,-0.02,-0.14,61.63,15.31,82.19,2.74,-9.51,1.21,14,5144,988.0,0.0


In [4]:
big_dataframe['FSR1'] = big_dataframe['FSR2'].astype(float)
big_dataframe['FSR2'] = big_dataframe['FSR2'].astype(float)

In [ ]:
# Create test set from all participants data

# Grouping by 'class' and 'participant'
grouped_dataframe = big_dataframe.groupby(['class', 'participant'])

# Function to select 16 random serial numbers and corresponding rows
def select_random_rows(group):
    # Sample 16 random serial numbers
    sampled_serial_numbers = group['serialnumber'].sample(n=17, replace=True, random_state=41).tolist()
    
    # Select all rows with the sampled serial numbers
    selected_rows = group[group['serialnumber'].isin(sampled_serial_numbers)]
    
    return selected_rows

# Apply the function to each group
test_set = grouped_dataframe.apply(select_random_rows).reset_index(drop=True)
test_set

In [ ]:
# Create train_set without the data in test_set

# Create a boolean mask to identify rows to be removed
mask = big_dataframe.isin(test_set.to_dict('list')).all(axis=1)
train_set = big_dataframe[~mask].reset_index(drop=True)

In [ ]:
unique_serial_numbers = train_set['serialnumber'].unique()
X_train = []
y_train = []

# Iterate through unique serial numbers and append arrays and class values to the lists
for serial_number in unique_serial_numbers:
    subset_df = train_set[train_set['serialnumber'] == serial_number]
    acc2x_y_array = subset_df[['FSR1', 'FSR2']].to_numpy()
    class_value = subset_df['class'].iloc[0]
    X_train.append(acc2x_y_array)
    y_train.append(class_value)

In [ ]:
unique_serial_numbers = test_set['serialnumber'].unique()

X_test = []
y_test = []

for serial_number in unique_serial_numbers:
    subset_df = test_set[test_set['serialnumber'] == serial_number]
    acc2x_y_array = subset_df[['gyro1','gyro2','gyro3']].to_numpy() # Change the features you want to train
    class_value = subset_df['class'].iloc[0]
    X_test.append(acc2x_y_array)
    y_test.append(class_value)

In [ ]:
from tslearn.utils import to_time_series_dataset
X_train = to_time_series_dataset(X_train)
X_test = to_time_series_dataset(X_test)


In [ ]:
!pip install tslearn
!pip install --upgrade scikit-learn

In [ ]:
from sklearn.model_selection import train_test_split

X = X_train
y = y_train

print("X_train size:", len(X_train))
print("X_test size:", len(X_test))
print("y_train size:", len(y_train))
print("y_test size:", len(y_test))

In [ ]:
import numpy
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn import metrics
import time
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneGroupOut

In [ ]:
#knn dtw
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from sklearn.metrics import accuracy_score

In [ ]:
tsc_dtw = KNeighborsTimeSeriesClassifier(n_neighbors=2, metric="dtw", n_jobs = -1)

# Perform k-fold cross-validation
k = 10
accuracy_scores = cross_val_score(tsc_dtw, X_train, y_train, cv=k)
average_accuracy = np.mean(accuracy_scores)

print("Accuracy scores for each fold:", accuracy_scores)
print("Average Accuracy:", average_accuracy)


In [ ]:
#training time
start_time = time.time()
tsc_dtw.fit(X_train, y_train)
training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

In [ ]:
import joblib

# Save the trained model to a file
joblib.dump(tsc_dtw, 'random_forest_model.joblib')


In [ ]:
start_time_single_prediction = time.time()
y_dtw = tsc_dtw.predict(X_test)
single_prediction_time = time.time() - start_time_single_prediction
print("Single Prediction time:", single_prediction_time/len(y_test), "seconds")

print("Accuracy using Knn: {0:.2f}".format(accuracy_score(y_dtw,y_test)))


In [ ]:
#svm
from tslearn.svm import TimeSeriesSVC
clf = TimeSeriesSVC(C=1.0, kernel="gak")

In [ ]:
#cv -svm

k = 10
accuracy_scores = cross_val_score(clf, X_train, y_train, cv=k)
average_accuracy = np.mean(accuracy_scores)

print("Accuracy scores for each fold:", accuracy_scores)
print("Average Accuracy:", average_accuracy)

In [ ]:
start_time = time.time()
clf.fit(X_train, y_train)

# Calculate the training time
training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

In [ ]:
start_time_single_prediction = time.time()
y_svm = clf.predict(X_test)#svm
single_prediction_time = time.time() - start_time_single_prediction
print("Single Prediction time:", single_prediction_time/len(y_test), "seconds")

print("Accuracy using SVM: {0:.2f}".format(accuracy_score(y_svm,y_test)))#svm